In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("train.csv")

In [3]:
X = df.drop("label", axis=1)
y = df["label"]

In [4]:
from sklearn.svm import SVC

In [5]:
from sklearn.linear_model import LogisticRegression

In [25]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [26]:
import optuna

In [27]:
from sklearn.model_selection import GridSearchCV

In [28]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import joblib
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [32]:
xgb_t = XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    random_state=42
)

In [33]:
params = {
    'n_estimators': np.arange(100, 800, 100),
    'max_depth': np.arange(3, 10),
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'subsample': np.linspace(0.6, 1.0, 5),
    'colsample_bytree': np.linspace(0.6, 1.0, 5)
}

In [34]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [35]:
random_search_x = RandomizedSearchCV(xgb_t, params, n_iter=10, scoring='neg_log_loss', cv=cv, verbose=1, n_jobs=-1)

In [36]:
random_search_x.fit(X, y)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='mlogloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=Non...
                   param_distributions={'colsample_bytree': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'learning_rate': array([0.01      , 0.04222222, 0.07444444, 0.10666667, 0.13888889,
       0.17111111, 0.20333333, 0.23555556, 0.26777778, 0.3       ]),
                                        'max_depth': array([3, 4, 5, 6, 7, 8, 9]),
                                        'n_estimators': array([100, 200, 300, 400, 500, 600, 700]),
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   scoring='neg_log_loss', verbose=1)

In [37]:
import numpy as np
np.unique(y)


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [38]:
best_xgb=random_search_x.best_params_

In [39]:
joblib.dump(best_xgb,"best_xgb.pkl")

['best_xgb.pkl']

In [40]:
lgb_t = LGBMClassifier(
    objective='multiclass',
    random_state=42,
    n_jobs=-1
)

In [41]:
params = {
    'n_estimators': np.arange(100, 800, 100),
    'max_depth': np.arange(3, 12),
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'num_leaves': np.arange(20, 100, 10),
    'subsample': np.linspace(0.6, 1.0, 5),
    'colsample_bytree': np.linspace(0.6, 1.0, 5),
}

In [42]:
random_search_l = RandomizedSearchCV(
    lgb_t,
    params,
    n_iter=10,
    scoring='neg_log_loss',
    cv=cv,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

In [43]:
random_search_l.fit(X, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010401 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4536
[LightGBM] [Info] Number of data points in the train set: 411230, number of used features: 20
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Warning] No further splits with p

RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=LGBMClassifier(n_jobs=-1, objective='multiclass',
                                            random_state=42),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'learning_rate': array([0.01      , 0.04222222, 0.07444444, 0.10666667, 0.13888889,
       0.17111111, 0.20333333, 0.23555556, 0.26777778, 0.3       ]),
                                        'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11]),
                                        'n_estimators': array([100, 200, 300, 400, 500, 600, 700]),
                                        'num_leaves': array([20, 30, 40, 50, 60, 70, 80, 90]),
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   random_state=42, scoring='neg_log_loss', verbose=1)

In [44]:
best_lgb=random_search_l.best_params_

In [45]:
joblib.dump(best_lgb,"best_lgb.pkl")

['best_lgb.pkl']

In [11]:
import joblib
best_xgb = joblib.load("best_xgb.pkl")
best_lg = joblib.load("best_lg.pkl")
best_lgb = joblib.load("best_lgb.pkl")

In [8]:
#model b training
lgb=LGBMClassifier(**best_lgb,random_state=42)
lgb.fit(X,y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056091 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4536
[LightGBM] [Info] Number of data points in the train set: 411230, number of used features: 20
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

LGBMClassifier(colsample_bytree=np.float64(1.0),
               learning_rate=np.float64(0.10666666666666666),
               max_depth=np.int32(9), n_estimators=np.int32(600),
               num_leaves=np.int32(70), random_state=42,
               subsample=np.float64(0.9))

In [9]:
joblib.dump(lgb,"lgb.pkl")

['lgb.pkl']

In [12]:

xgb=XGBClassifier(**best_xgb,random_state=42)
xgb.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=np.float64(0.8), device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None,
              learning_rate=np.float64(0.1711111111111111), max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=np.int32(8), max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=np.int32(700), n_jobs=None,
              num_parallel_tree=None, ...)

In [13]:
joblib.dump(xgb,"xgb.pkl")

['xgb.pkl']

In [50]:
print(X.shape)

(411230, 20)


In [51]:
lg = LogisticRegression(max_iter=100, solver='liblinear')

In [52]:
param_dist_lg = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3, 3, 20),       
    'solver': ['saga']   
}

In [53]:
random_search_lr = RandomizedSearchCV(
    estimator=lg,
    param_distributions=param_dist_lg,
    n_iter=20,                 
    scoring='accuracy',        
    n_jobs=-1,
    cv=cv,
    random_state=42,
    verbose=1
)

In [54]:
random_search_lr.fit(X, y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\Aleena\anaconda3\New folder\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=LogisticRegression(solver='liblinear'), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03]),
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['saga']},
                   random_state=42, scoring='accuracy', verbose=1)

In [55]:
best_lg=random_search_lr.best_params_

In [56]:
joblib.dump(best_lg,"best_lg.pkl")

['best_lg.pkl']

In [15]:
best_lg=joblib.load("best_lg.pkl")

In [16]:

lg=LogisticRegression(**best_lg,random_state=42)
lg.fit(X,y)

c:\Users\Aleena\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=np.float64(233.57214690901213), penalty='l1',
                   random_state=42, solver='saga')

In [17]:
joblib.dump(lg,"lg.pkl")

['lg.pkl']

In [18]:
#svm = SVC(probability=True)

In [19]:
# param_dist_svm = {
#     'C': np.logspace(-3, 3, 20),   # regularization parameter
#     'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
#     'gamma': ['scale', 'auto']
# }

In [20]:
# random_search_svm = RandomizedSearchCV(
#     estimator=svm,
#     param_distributions=param_dist_svm,
#     n_iter=20,
#     scoring='accuracy',
#     n_jobs=-1,
#     cv=cv,
#     random_state=42,
#     verbose=1
# )